In [3]:
import pandas as pd
df = pd.read_csv("pre_processed_50k.csv")

In [5]:
df.head()

,Unnamed: 0,label,uuid,title,text,pre_processed_text
0,317840,Sports,8aa262b970291df4538fb81ed00fc17362ba0808,DeGrom outduels Kershaw with 13 Ks; Mets beat ...,> Sports DeGrom outduels Kershaw with 13 Ks; M...,"['sports', 'degrom', 'outduels', 'kershaw', 't..."
1,210418,Sports,4d966e226658b7172bc77648cd24c8486ce5435c,Packers hold off pass-happy Chargers for 27-20...,"Packers overcome big day by Rivers, hold off p...","['packers', 'overcome', 'big', 'day', 'rivers'..."
2,314625,Sports,3ed20bb17dbdc4af9cf28559bf585a5764f231c3,"Special athletes humiliated, passports confisc...",Saina Nehwal gifts her 'racquet' to PM Narendr...,"['saina', 'nehwal', 'gifts', 'racquet', 'pm', ..."
3,332182,Sports,1f76f7f913fcc274a1408b3f72ce54ceb6120085,Tony the Tiger helps a prostitute in hoax camp...,Tony is Back!/YouTube A moment from a video fe...,"['tony', 'back', 'youtube', 'moment', 'video',..."
4,241437,Sports,f4202e88343ee2a6d6c3d0507d4b4d24f5c59688,"Jose Mourinho attacks 'balding' Guardiola, 'ru...",Date: 08-10-2015 Time: 03:10:40:pm Mourinho \n...,"['date', 'eight million, one hundred and two t..."


In [6]:
%%time
import io
def load_vectors(fname):
    fin = io.open(fname, 'r', encoding='utf-8', newline='\n', errors='ignore')
    n, d = map(int, fin.readline().split())
    data = {}
    for line in fin:
        tokens = line.rstrip().split(' ')
        data[tokens[0]] = list(map(float, tokens[1:]))
    return n,d,data

n,d,ft_dict = load_vectors("assets/wiki-news-300d-1M.vec")

In [41]:
import numpy as np
def get_ft_vec(token,ft_dict,d=300):
    try:
        v = np.array(ft_dict[token])
    except KeyError:
        v = np.array([0]*d)
    return v

def compute_ft_sum(textlist,ft_dict=ft_dict,d=300):
    ret_vec = np.array([0]*d)
    for key in textlist:
        # Multiply the word embedding with the 
        # number of times the user searched 
        vec = get_ft_vec(key,ft_dict)
        # Vector addition of token embeddings
        ret_vec = ret_vec + vec
    return list(ret_vec)


In [ ]:
# get_ft_vec("", ft_dict)
compute_ft_sum(["Asdf", "Asdf"], ft_dict)

In [44]:
%%time
# (df.head())
# df.text
from multiprocessing import Pool

pool = Pool()
results = pool.map(compute_ft_sum, list(df.pre_processed_text))
pool.close()


CPU times: user 7.46 s, sys: 2.39 s, total: 9.85 s
Wall time: 6min 56s


In [49]:
thattable = pd.DataFrame(results)

In [51]:
thattable["label"] = df.label

In [54]:
thattable.to_csv("labelled_vector_50000cross301.csv")

In [58]:
thattable[thattable["label"]=="Sports"]

,0,1,2,3,4,5,6,7,8,9,...,291,292,293,294,295,296,297,298,299,label
0,-198.8716,25.9461,-199.0322,-77.2522,103.4592,42.1401,66.4375,-78.8471,212.7611,87.3641,...,48.6905,-26.7277,97.5663,52.4087,37.1607,28.5061,-55.3705,-7.7584,6.0376,Sports
1,-193.7087,19.6114,-261.3492,-75.5626,126.0257,32.0915,104.1218,-121.7503,234.9799,100.9158,...,68.4136,-72.8238,113.1230,56.8972,24.1610,18.5288,-118.5269,-0.0650,-25.8585,Sports
2,-141.2991,16.3996,-160.2514,-55.7691,90.9411,19.3411,71.4529,-78.3108,149.9634,58.5939,...,52.4711,-46.3255,80.5317,35.8761,19.6296,4.4644,-59.0167,7.9477,-17.8971,Sports
3,-28.0998,3.7503,-34.3880,-7.0165,17.2936,1.7903,14.5773,-15.6619,29.9520,13.4891,...,11.1246,-11.1175,16.4866,11.6771,3.4959,2.4954,-14.1479,0.2278,-3.4901,Sports
4,-193.2378,16.7498,-233.2637,-62.9471,130.8930,24.6702,102.3759,-91.4669,206.5019,92.8209,...,65.4048,-57.1671,102.4619,63.3899,27.9555,24.0707,-99.7016,21.7909,-17.6547,Sports
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,-74.9502,9.0651,-80.1342,-29.8325,44.4700,19.1447,28.5879,-33.9763,83.0439,33.8532,...,17.0112,-12.4829,39.4216,18.6478,13.2685,8.0718,-27.9293,-5.5294,-0.7787,Sports
9996,-246.6059,32.2269,-308.3488,-90.1695,166.3803,29.2560,133.8592,-132.9726,286.5101,121.8671,...,87.6733,-91.9705,146.8783,66.3574,28.2196,35.8524,-123.4147,4.6124,-31.4373,Sports
9997,-281.7603,40.9292,-336.9405,-98.2962,206.7021,34.5163,151.1572,-136.9232,318.7536,149.4891,...,97.2350,-81.7114,163.0455,80.1949,41.4704,35.6713,-157.8098,29.3300,-32.4486,Sports
9998,-13.9315,-1.7492,-20.0259,-5.6044,8.3774,4.0673,7.7535,-9.3679,15.6640,6.6407,...,3.5713,-3.6052,6.9852,5.8730,2.1569,2.0103,-8.1061,-1.2584,-0.1159,Sports


In [59]:
thattable[thattable["label"]=="Entertainment"]

,0,1,2,3,4,5,6,7,8,9,...,291,292,293,294,295,296,297,298,299,label
20000,-143.8041,22.4040,-158.4257,-58.3176,91.4216,15.8475,71.8961,-72.4728,155.2001,59.1021,...,52.3601,-40.6831,82.4861,29.1630,24.2205,13.4441,-52.8053,12.3034,-14.0434,Entertainment
20001,-202.7640,28.3398,-254.2037,-79.2248,146.7881,36.9738,103.3447,-113.3218,229.3480,107.1050,...,67.8800,-57.8066,110.9729,60.8311,33.8461,16.8385,-123.5500,18.8052,-23.3602,Entertainment
20002,-83.9313,11.6364,-105.8013,-32.3744,55.3888,14.0653,45.2146,-44.9588,99.5099,42.5421,...,29.9283,-26.4713,51.1948,23.5086,12.7943,11.2850,-49.2853,4.6439,-6.3497,Entertainment
20003,-78.5002,8.9641,-93.2702,-36.7406,56.8240,14.6611,41.2055,-41.2480,84.7219,35.6408,...,25.4983,-22.5023,43.0240,20.5067,9.3982,5.2934,-41.1875,5.4094,-7.7277,Entertainment
20004,-65.2442,10.5913,-83.4893,-31.2640,44.7305,11.3249,34.3262,-41.6667,79.4709,34.1173,...,27.3907,-24.2117,39.9716,19.0853,9.9240,0.2517,-37.0872,4.3310,-12.0578,Entertainment
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29995,-80.5161,10.4965,-105.4058,-29.3735,50.0795,13.3064,42.9630,-46.1213,98.0188,45.1885,...,39.5712,-32.0978,51.0645,33.6491,7.1347,2.9292,-48.9233,5.6556,-8.5313,Entertainment
29996,-137.9836,10.4829,-160.9993,-57.1396,86.3784,30.2929,58.3515,-79.8048,153.7387,60.2624,...,49.0214,-43.5789,76.7630,35.1697,12.9439,7.4810,-60.1122,-11.6888,-13.3818,Entertainment
29997,-300.1680,54.4281,-372.6858,-125.5019,214.9448,43.2504,172.6188,-165.9622,355.0680,158.4613,...,109.3516,-91.4557,185.3255,78.9336,53.3704,26.1164,-175.3988,31.5094,-39.1039,Entertainment
29998,-81.3320,11.1742,-102.5507,-31.4173,54.6060,13.4989,42.2514,-47.2240,92.8535,40.5204,...,27.3003,-28.9741,47.6179,18.6713,12.1846,11.6808,-41.4792,6.9410,-13.3851,Entertainment


In [81]:
thattable

,0,1,2,3,4,5,6,7,8,9,...,291,292,293,294,295,296,297,298,299,label
0,-198.8716,25.9461,-199.0322,-77.2522,103.4592,42.1401,66.4375,-78.8471,212.7611,87.3641,...,48.6905,-26.7277,97.5663,52.4087,37.1607,28.5061,-55.3705,-7.7584,6.0376,Sports
1,-193.7087,19.6114,-261.3492,-75.5626,126.0257,32.0915,104.1218,-121.7503,234.9799,100.9158,...,68.4136,-72.8238,113.1230,56.8972,24.1610,18.5288,-118.5269,-0.0650,-25.8585,Sports
2,-141.2991,16.3996,-160.2514,-55.7691,90.9411,19.3411,71.4529,-78.3108,149.9634,58.5939,...,52.4711,-46.3255,80.5317,35.8761,19.6296,4.4644,-59.0167,7.9477,-17.8971,Sports
3,-28.0998,3.7503,-34.3880,-7.0165,17.2936,1.7903,14.5773,-15.6619,29.9520,13.4891,...,11.1246,-11.1175,16.4866,11.6771,3.4959,2.4954,-14.1479,0.2278,-3.4901,Sports
4,-193.2378,16.7498,-233.2637,-62.9471,130.8930,24.6702,102.3759,-91.4669,206.5019,92.8209,...,65.4048,-57.1671,102.4619,63.3899,27.9555,24.0707,-99.7016,21.7909,-17.6547,Sports
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49995,-460.9932,64.4113,-505.6208,-188.5262,292.1669,87.5246,198.0076,-243.9398,501.1492,206.1962,...,160.4215,-117.1199,252.4607,129.3807,81.3499,21.6068,-206.4776,26.0155,-47.2231,Tech
49996,-150.6876,20.3695,-184.9659,-54.0366,120.1983,30.4444,92.4437,-84.3667,188.6445,72.9943,...,52.9908,-46.6870,92.2219,50.3538,28.2423,4.0089,-92.0411,2.6526,-23.4992,Tech
49997,-502.5111,61.4077,-542.7087,-222.9414,323.0772,100.1651,223.5054,-250.3312,533.5650,215.7267,...,158.3086,-114.3705,279.7248,111.6568,88.3206,24.9333,-209.0775,15.3221,-37.3990,Tech
49998,-134.5173,10.2441,-154.2745,-53.5732,76.8308,23.3325,55.2890,-77.7074,140.6295,60.0086,...,46.1830,-36.0710,70.2037,40.1052,21.9463,3.5900,-52.1954,2.7055,-12.8917,Tech


In [89]:
# for l in ['Sports', 'Politics', 'Entertainment', 'Finance', 'Tech']:
#     print(thattable[thattable["label"]==l][0:10])

import numpy as np
# thattable.drop("label", axis=1)
# thattable
sq = np.square(thattable.drop("label", axis=1))
sqrrt = np.sqrt(sq.sum(axis=0))

final = thattable.drop("label", axis=1).divide(sqrrt, axis=1)


0      1.0
1      1.0
2      1.0
3      1.0
4      1.0
      ... 
295    1.0
296    1.0
297    1.0
298    1.0
299    1.0
Length: 300, dtype: float64

In [93]:
final["labal"] = thattable["label"]

In [97]:
final.to_csv("final_unit_vector.csv")

In [109]:
# for l in ['Sports', 'Politics', 'Entertainment', 'Finance', 'Tech']:
#     print(final[final["labal"]==l][0:10])

final[final["labal"]=="Sports"].head()

,0,1,2,3,4,5,6,7,8,9,...,291,292,293,294,295,296,297,298,299,labal
0,-0.003003,0.003249,-0.002608,-0.002868,0.002543,0.003232,0.002214,-0.002242,0.002862,0.002786,...,0.002107,-0.001440,0.002618,0.002592,0.003740,0.006448,-0.001713,-0.001727,0.000886,Sports
1,-0.002925,0.002456,-0.003424,-0.002806,0.003098,0.002462,0.003469,-0.003462,0.003161,0.003218,...,0.002961,-0.003923,0.003035,0.002814,0.002432,0.004191,-0.003667,-0.000014,-0.003793,Sports
2,-0.002133,0.002053,-0.002100,-0.002071,0.002236,0.001484,0.002381,-0.002227,0.002017,0.001868,...,0.002271,-0.002495,0.002161,0.001774,0.001975,0.001010,-0.001826,0.001769,-0.002625,Sports
3,-0.000424,0.000470,-0.000451,-0.000261,0.000425,0.000137,0.000486,-0.000445,0.000403,0.000430,...,0.000481,-0.000599,0.000442,0.000577,0.000352,0.000564,-0.000438,0.000051,-0.000512,Sports
4,-0.002918,0.002097,-0.003056,-0.002337,0.003218,0.001892,0.003411,-0.002601,0.002778,0.002960,...,0.002831,-0.003079,0.002749,0.003135,0.002813,0.005445,-0.003085,0.004850,-0.002590,Sports


In [108]:
final[final["labal"]=="Entertainment"].head()

,0,1,2,3,4,5,6,7,8,9,...,291,292,293,294,295,296,297,298,299,labal
20000,-0.002171,0.002805,-0.002076,-0.002165,0.002247,0.001216,0.002395,-0.002061,0.002088,0.001885,...,0.002266,-0.002191,0.002213,0.001442,0.002438,0.003041,-0.001634,0.002738,-0.002060,Entertainment
20001,-0.003061,0.003548,-0.003330,-0.002942,0.003609,0.002836,0.003443,-0.003223,0.003085,0.003415,...,0.002938,-0.003114,0.002978,0.003008,0.003406,0.003809,-0.003823,0.004186,-0.003427,Entertainment
20002,-0.001267,0.001457,-0.001386,-0.001202,0.001362,0.001079,0.001506,-0.001279,0.001339,0.001357,...,0.001295,-0.001426,0.001374,0.001162,0.001288,0.002553,-0.001525,0.001034,-0.000931,Entertainment
20003,-0.001185,0.001122,-0.001222,-0.001364,0.001397,0.001125,0.001373,-0.001173,0.001140,0.001137,...,0.001104,-0.001212,0.001154,0.001014,0.000946,0.001197,-0.001274,0.001204,-0.001134,Entertainment
20004,-0.000985,0.001326,-0.001094,-0.001161,0.001100,0.000869,0.001144,-0.001185,0.001069,0.001088,...,0.001185,-0.001304,0.001073,0.000944,0.000999,0.000057,-0.001148,0.000964,-0.001769,Entertainment
